# Introduction
This notebook shows how to create a simple but very silly strategy from scratch, the ** Werewolf Strategy **.

Our strategy adheres to the following 3 rules: 

1. If it is a super moon => we buy stocks
2. If it is a micro moon => we sell our stocks so we have no more exposure
3. Otherwise => we do nothing

Btw, a supermoon occurs when the moon’s orbit is closest to Earth at the same time it is full. So visually it is a very large moon. 

The main purpose of this example is obviously not to create a useful strategy that will make you a huge profit. But rather to demonstrate how easy it is to develop your own strategy.
As always, we start with loading roboquant using the cell magic *%use*


In [ ]:
%use roboquant(1.4.0)
Welcome()

## Load external libraries
We can load any library from external repositories and import the required packages afterwards. If required we can even add repositories to the notebook. In the background the notebook kernel will download the required package and all its declared dependencies. 

In this case we download the *common-suncalc* package for calculating the moon phases. And because it is available in one of the standard Maven repositories, we don't need to add that first.  

In [ ]:
@file:DependsOn("org.shredzone.commons:commons-suncalc:3.4")
import org.shredzone.commons.suncalc.*

## Coding our own Strategy

We're now ready to develop the custom strategy. In this case we subclass the convenience class *SingleAssetStrategy* that can be used for strategies that are only interested in a single asset at a time. We need to override the method with signature:

```kotlin
fun generate(priceAction: PriceAction, now: Instant): Signal?
```
    
We get as input the price action for that asset and a timestamp for when the event occurred. We return a Signal that indicates if we want to buy or sell that asset. We return *null* if we don't want to do anything. So the whole custom strategy is just a few lines of code.

In [ ]:
class WerewolfStrategy(asset: Asset) : SingleAssetStrategy(asset) {

    override fun generate(priceAction: PriceAction, now: Instant): Signal? {
        val phase = MoonPhase.compute().on(now).execute()
        return when {
            phase.isSuperMoon -> Signal(asset, Rating.BUY)
            phase.isMicroMoon -> Signal(asset, Rating.SELL)
            else -> null
        }
    }
    
}

## Create a Roboquant instance and evaluate it
We are now ready to back-test our strategy. And we decided to test it against Apple stock. So what we need to do is:

- Use a data feed that contains historic prices for the Apple stock
- Create an instance of our strategy and tell it to focus on the Apple stock
- Create an instance of a roboquant and run it against the available data 

In [ ]:
val feed = AvroFeed.sp500()
val apple = feed.assets.getBySymbol("AAPL")
val strategy = WerewolfStrategy(apple)
val roboquant =  Roboquant(strategy)

roboquant.run(feed)

## Get some insights
Let's start to see how the account looks like after the experiment has run. The account holds the latest state

In [ ]:
val account = roboquant.broker.account
account.summary()

By default, the simulated broker account contains an initial cash deposit of 1_000_000.00 USD. Lets see how much realized profit we actually made with our trades:

In [ ]:
account.trades.realizedPNL

## Lessons learned

* It is not that hard to develop a new strategy and back-test it. 
* Even the most silly strategies can **appear** to make a profit if back-tests are based on unsound assumptions. In this case we are running our back test solely against Apple stocks and only going long; we were almost sure to make a decent profit.  